In [1]:
from sapphire import *

    Management IP   Password
0   10.127.119.83   nbv_3124
1  10.127.119.103   nbv_3124
2   10.127.119.84   nbv_3124
3   10.127.119.98   nbv_3124
4  10.127.125.228  nbv_12345


In [2]:
mgmt_ip = '10.127.125.219'

mgmt_ip = '10.127.190.134'
password = 'nbv_12345'

switch = Switch(mgmt_ip, password)
vsan = 200

Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fbd7b8c0bb0>
Switch object created with paramiko client in-built.


In [3]:
switch

In [4]:
fabric = Fabric(switch)

Fabric object initialized


In [5]:
fabric.search_devices(vsan = vsan)

In [6]:
zone_dict = switch.get_zoneset_active(vsan = vsan)
print(zone_dict)

Getting active zoneset info for vsan 200
Active Zoneset : SaintArcher_zoneset2
{'emulex_host': [('fcid', '0xa10000'), ('fcid', '0xa100ef')], 'cisco_10G_host': [('fcid', '0xa10021'), ('fcid', '0xa100ef')], 'cisco_40G_host': [('fcid', '0xa10001'), ('fcid', '0xa100ef')], 'cisco_25G_host': [('fcid', '0xa10040'), ('fcid', '0xa100ef')]}


In [7]:
zone_df = pd.DataFrame.from_dict(zone_dict, orient = 'index')
zone_df.head(df.shape[0])

,0,1
emulex_host,"(fcid, 0xa10000)","(fcid, 0xa100ef)"
cisco_10G_host,"(fcid, 0xa10021)","(fcid, 0xa100ef)"
cisco_40G_host,"(fcid, 0xa10001)","(fcid, 0xa100ef)"
cisco_25G_host,"(fcid, 0xa10040)","(fcid, 0xa100ef)"


In [8]:
flogi_df = switch.get_flogi_database(vsan)

In [9]:
switch.show_flogi_database(vsan)

             Interface VSAN      FCID                PORT NAME  \
0               fc1/13  200  0xa100ef  50:06:01:68:3e:e0:20:f9   
1  san-port-channel211  200  0xa10000  10:00:00:10:9b:22:5b:4d   
2  san-port-channel211  200  0xa10021  20:00:7c:31:0e:8f:c7:c9   
3  san-port-channel212  200  0xa10040  20:00:7c:31:0e:8f:c7:cb   
4               vfc213  200  0xa10001  20:00:a8:b4:56:50:bc:be   

                 NODE NAME  
0  50:06:01:60:be:e0:20:f9  
1  20:00:00:10:9b:22:5b:4d  
2  10:00:7c:31:0e:8f:c7:c9  
3  10:00:7c:31:0e:8f:c7:cb  
4  10:00:a8:b4:56:50:bc:be  


In [10]:
flogi_df.loc[flogi_df['FCID'] == '0xa10040']['Interface'].values

array(['san-port-channel212'], dtype=object)

In [11]:
# See https://neo4j.com/developer/aura-connect-driver/ for Aura specific connection URL.
scheme = "neo4j"  # Connecting to Aura, use the "neo4j+s" URI scheme
host_name = "localhost"
port = 7687
url = "{scheme}://{host_name}:{port}".format(scheme=scheme, host_name=host_name, port=port)
user = "admin"
password = "admin"
app = App(url, user, password)

In [12]:
def get_switch_port(fabric, fcid):
    #print('Getting switch_port for fcid {}'.format(fcid))
    for switch in fabric.devices:
        #print('Searching for FCID {} in switch {}'.format(fcid, switch.switchname))
        flogi_df = switch.get_flogi_database(vsan)
        port = flogi_df.loc[flogi_df['FCID'] == fcid]['Interface'].values
        if len(port) == 1:
            #print('FCID found')
            switch_port = switch.switchname + '_' + port[0]
            break
        else:
            #print('FCID {} not found, going to next switch...')
            continue
    return switch_port
    
for index, row in zone_df.iterrows(): 
    print('Iterating over devices under zone {}'.format(index))
    concerned_ports = []
    for i in range(len(row)):
        if row[i][0] == 'fcid':
            switch_port = get_switch_port(fabric, row[i][1])
            #print(row[i][1], switch_port)
            concerned_ports.append(switch_port)
        elif row[i][1] == 'pwwn':
            pass
        elif row[i][1] == 'fwwn':
            pass
        elif row[i][1] == 'device-alias':
            pass
        elif row[i][1] == 'fcalias':
            pass
    
    #Now, create links between all the devices under the same zone.
    for i in range(len(concerned_ports)):
        for j in range(i+1, len(concerned_ports)):
            print('Creating virtual link between {} and {}'.format(concerned_ports[i], concerned_ports[j]))
            app.create_link(concerned_ports[i], concerned_ports[j], 'V')
            
    
            

Iterating over devices under zone emulex_host
Creating virtual link between HW-Core-2_san-port-channel211 and HW-Core-2_fc1/13
Iterating over devices under zone cisco_10G_host
Creating virtual link between HW-Core-2_san-port-channel211 and HW-Core-2_fc1/13
Iterating over devices under zone cisco_40G_host
Creating virtual link between HW-Core-2_vfc213 and HW-Core-2_fc1/13
Iterating over devices under zone cisco_25G_host
Creating virtual link between HW-Core-2_san-port-channel212 and HW-Core-2_fc1/13


In [13]:
app.close()